In [1]:
import json
import numpy as np
from numpy import linalg as LA
import torch
def load_json(filename):
    with open(f"{filename}", 'r') as fp:
        data = json.load(fp)
    return data
def save_to_json(data, filename):
    with open(f"{filename}", 'w') as fp:
        json.dump(data, fp)

In [2]:
if torch.cuda.is_available():  
    print("Cuda available")
    dev = "cuda:0" 
else:  
    dev = "cpu"  
device = torch.device(dev)  
a = torch.zeros(4,3)    
a = a.to(device)

Cuda available


In [3]:
resources = '/home/nikolaisv/study/intership/work/technical-connections/resources/'
idToCommit = load_json(f"{resources}/idToCommit")
size = len(idToCommit)
shape = (size, size)
concurrentGraph = load_json(f"{resources}/concurrentGraph")
alpha = 0.85

In [4]:
H = torch.zeros(shape)
A = torch.full(shape, 1 / size)
O = torch.ones(shape)

H.to(device)
A.to(device)
O.to(device)

for key, value in concurrentGraph.items():
    k = int(key)
    for adjCommit in value:
        commit = int(adjCommit)
        H[k][commit] = 1
    if len(value) != 0:
        A[:, k] = 0

In [5]:
G = alpha * H + alpha * A + (1 - alpha) * (1 / size) * O
G.to(device)

tensor([[7.3298e-05, 7.3298e-05, 1.0995e-05,  ..., 7.3298e-05, 1.0995e-05,
         7.3298e-05],
        [7.3298e-05, 7.3298e-05, 1.0995e-05,  ..., 7.3298e-05, 1.0995e-05,
         7.3298e-05],
        [7.3298e-05, 8.5007e-01, 1.0995e-05,  ..., 7.3298e-05, 1.0995e-05,
         7.3298e-05],
        ...,
        [7.3298e-05, 7.3298e-05, 1.0995e-05,  ..., 7.3298e-05, 1.0995e-05,
         7.3298e-05],
        [7.3298e-05, 7.3298e-05, 1.0995e-05,  ..., 7.3298e-05, 1.0995e-05,
         7.3298e-05],
        [7.3298e-05, 7.3298e-05, 1.0995e-05,  ..., 7.3298e-05, 1.0995e-05,
         7.3298e-05]], device='cuda:0')

In [6]:
G.max(), G.dtype

(tensor(0.8501), torch.float32)

In [7]:
I = torch.zeros((G.shape[0], 1), dtype=torch.float32)
I[0][0] = 1
I.to(device)
for _ in range(100):
    I = G @ I

In [8]:
result = I / I.max()
result

tensor([[0.0007],
        [0.0007],
        [0.0011],
        ...,
        [0.0007],
        [0.0011],
        [0.0007]])

In [10]:
save_to_json(result.tolist(), f"{resources}/commitsIncome")

In [53]:
from pyvis.network import Network
class Graph:
    """
    Class for creating file congruence graph.
    Nodes representing users. Edge exists between nodes if users got common changed files. Edge color and width depends
    on number of common files between users.
    """

    def __init__(self, M, node_size, mult=1, heading="Graph"):
        self.M = M
        self.mult = mult
        self.node_size = node_size

        self.__init_net(heading)


    def __init_net(self, heading):
        """
        Create net-graph
        :param heading: heading
        """
        self.net = Network(heading=heading
                           , height="1080px"
                           , width="100%"
                           , bgcolor="#222222"
                           , font_color="white")

        
        self.net.barnes_hut()

        n=0
        non_zeros = torch.nonzero(self.M.view(-1).data).squeeze()
        non_zeros = torch.nonzero(self.M.view(-1).data).squeeze()
        _I = non_zeros // self.M.shape[0]
        _J = non_zeros % self.M.shape[0]
        
        for i,j in zip(_I, _J):

            weight = self.M[i][j]
            print(f"{i} {j} = {weight}")
            if weight == 0:
                continue
            src = str(i.item())
            dst = str(j.item())

#                 print(f"{src} -> {dst} = {weight}")
            self.net.add_node(src, src, title=src, color="red", size=self.node_size[i].item()*10000)
            self.net.add_node(dst, dst, title=dst, color="blue", size=self.node_size[j].item()*10000)
            self.net.add_edge(src, dst, value=int(weight), color="yellow")
            n+=1
            if n > 100:
                break



    def show(self):
        print(f"{self.net.heading}.html")
        self.net.show(f"{self.net.heading}.html")


    @staticmethod
    def edge_color(weight, quantile1, quantile2, quantile3):
        if weight <= quantile1:
            return '#0569E1'
        if weight <= quantile2:
            return '#C1F823'
        if weight <= quantile3:
            return '#FCAA05'
        return '#EE5503'

In [20]:
non_zeros = torch.nonzero(H.view(-1).data).squeeze()

In [28]:
non_zeros[0] // H.shape[0], non_zeros[0] % H.shape[0]

(tensor(2), tensor(1))

In [27]:
H[2][1]

tensor(1.)

In [44]:
H.mean()

tensor(9.0672e-05)

In [54]:
graph = Graph(H, result, heading="PageRank")
graph.show()

2 1 = 1.0
3 1 = 1.0
4 1 = 1.0
6 1 = 1.0
10 1 = 1.0
10 8 = 1.0
20 1 = 1.0
23 1 = 1.0
26 1 = 1.0
26 25 = 1.0
28 1 = 1.0
28 26 = 1.0
28 27 = 1.0
34 1 = 1.0
47 1 = 1.0
47 46 = 1.0
54 1 = 1.0
54 52 = 1.0
72 1 = 1.0
76 1 = 1.0
76 42 = 1.0
90 49 = 1.0
90 89 = 1.0
97 1 = 1.0
134 132 = 1.0
155 1 = 1.0
172 140 = 1.0
172 143 = 1.0
172 152 = 1.0
174 1 = 1.0
180 179 = 1.0
184 162 = 1.0
192 191 = 1.0
195 191 = 1.0
203 1 = 1.0
203 73 = 1.0
203 141 = 1.0
203 163 = 1.0
203 201 = 1.0
219 1 = 1.0
223 177 = 1.0
224 213 = 1.0
224 223 = 1.0
225 177 = 1.0
225 224 = 1.0
245 1 = 1.0
245 244 = 1.0
246 1 = 1.0
254 1 = 1.0
254 140 = 1.0
254 142 = 1.0
254 143 = 1.0
254 145 = 1.0
254 172 = 1.0
254 176 = 1.0
254 216 = 1.0
256 188 = 1.0
261 1 = 1.0
263 1 = 1.0
263 140 = 1.0
263 179 = 1.0
263 251 = 1.0
263 254 = 1.0
264 1 = 1.0
264 261 = 1.0
264 263 = 1.0
265 1 = 1.0
265 140 = 1.0
265 179 = 1.0
265 251 = 1.0
265 254 = 1.0
271 242 = 1.0
271 270 = 1.0
290 284 = 1.0
291 1 = 1.0
291 289 = 1.0
295 153 = 1.0
295 212 = 1.0
2

In [140]:
test_H = torch.tensor(
    [
    [ 0, 0, 0, 0, 0, 0, 1/3, 0 ],
    [ 1/2, 0, 1/2, 1/3, 0, 0, 0, 0 ],
    [ 1/2, 0, 0, 0, 0, 0, 0, 0 ],
    [ 0, 1, 0, 0, 0, 0, 0, 0 ],
    [ 0, 0, 1/2, 1/3, 0, 0, 1/3, 0 ],
    [ 0, 0, 0, 1/3, 1/3, 0, 0, 1/2 ],
    [ 0, 0, 0, 0, 1/3, 0, 0, 1/2 ],
    [ 0, 0, 0, 0, 1/3, 1, 1/3, 0 ]
    ],dtype=torch.float64, device=device
)

In [152]:
# I = torch.tensor([1,0,0,0,0,0,0,0], dtype=torch.float64, device=device)
I = torch.tensor([[1],[0],[0],[0],[0],[0],[0],[0]], dtype=torch.double, device=device)
for i in range(61):
    I = test_H @ I
#     print(f"itteration {i}")
I

tensor([[0.0600],
        [0.0675],
        [0.0300],
        [0.0675],
        [0.0975],
        [0.2025],
        [0.1800],
        [0.2950]], device='cuda:0', dtype=torch.float64)